### Concept Bottleneck on Microbiome Trajectories

This code is almost the same as for the plain transformer. The only difference in our loader is that we also need to keep track of the dense concept annotations, which are saved in to the `concepts` DataFrame. As before, you can download the data from [this link](https://github.com/krisrs1128/interpretability_review/tree/main/data).

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Subset
from concept import ConceptData

# use the data from ../generate
torch.manual_seed(20240210)
subsample_size = 500
samples_df = pd.read_csv("../data/blooms.csv", nrows=subsample_size)
concepts = pd.read_csv("../data/concepts.csv", nrows=subsample_size)

dataset = ConceptData(samples_df, concepts)
train = Subset(dataset, torch.arange(int(0.75 * subsample_size)))
validation = Subset(dataset, torch.arange(int(0.75 * subsample_size), subsample_size))
loaders = {
  "train": DataLoader(train, batch_size=16),
  "validate": DataLoader(validation, batch_size=16)
}

We are training the model for longer, because the concept model takes longer to converge. You can view all the evaluation metrics by starting a tensorboard viewer in the `concept_logs` directory created by the block below.

In [2]:
import lightning as L
from concept import ConceptBottleneck, LitConcept

concepts
model = ConceptBottleneck()
lit_model = LitConcept(model)
trainer = L.Trainer(max_epochs=70, default_root_dir="concept_logs")
trainer.fit(lit_model, loaders["train"], loaders["validate"])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params
--------------------------------------------
0 | model | ConceptBottleneck | 8.9 M 
--------------------------------------------
8.9 M     Trainable params
0         Non-trainable params
8.9 M     Total params
35.723    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/miniconda3/envs/interpretability/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import torch

lit_model.model.eval()
p_hat = []
with torch.no_grad():
  for x, c, _ in loaders["train"]:
    p_hat.append(lit_model.model(x)[1])
  for x, c, _ in loaders["validate"]:
    p_hat.append(lit_model.model(x)[1])

pd.DataFrame(torch.concatenate(p_hat)).to_csv("../data/p_hat_concept.csv")